In [231]:
%load_ext autoreload
%autoreload 2

import networkx as nx
import random
from typing import Dict, List, Tuple, Set
from problem import *
from round import *
from heuristics import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
client_locations_excluded = []
for person in client_locations:
    if len(person)>1:
        client_locations_excluded.append(person[1:])
        #What happens to the indexing of people then? Do we have to make a map?

#Brute force combinations
covered_client_locations = set()
client_locations = [[0, 1],[1,2],[3, 4]]

initial_guess = [loc[0] for loc in client_locations]
guesses = [([client_locations[ind][0] if ind!=0 else client_locations[ind][k] 
             for ind in range(len(client_locations))], 1)
           for k in range(len(client_locations[0]))]

while len(guesses)>0:
    guess, guess_client_level = guesses.pop()
    covered_client_locations.add(frozenset(guess))
    print(guess)
    
    if guess_client_level<len(client_locations):
        for ind in range(1, len(client_locations[guess_client_level])):
            loc = client_locations[guess_client_level][ind]
            new_guess = [i for i in guess]
            new_guess[guess_client_level] = loc

            if frozenset(new_guess) not in covered_client_locations:
                guesses.append((new_guess, guess_client_level+1))

[1, 1, 3]
[1, 2, 3]
[1, 2, 4]
[0, 1, 3]
[0, 2, 3]
[0, 2, 4]


In [149]:
client_locations_excluded = []
for person in client_locations:
    if len(person)>1:
        client_locations_excluded.append(person[1:])
        #What happens to the indexing of people then? Do we have to make a map?

covered_client_locations = set()
client_locations = [[0, 1],[1,2],[3, 4]]

#Generate the possible facility combinations
initial_guess = [loc[0] for loc in client_locations]
guesses = [[loc] for loc in client_locations[0]]

is_full = False
while not is_full:
    guess = guesses.pop(0)
    if len(guess) == len(client_locations):
        is_full = True
    else:
        ind = len(guess)
        for loc in client_locations[ind]:
            new_guess = [i for i in guess] + [loc]
            guesses.append(new_guess)

#Iterate through the possibilities and 
covered_dict = {}
for guess in guesses:
    set_guess = frozenset(guess)
    if set_guess not in covered_set:
        X = _k_supplier(distances, list(set(guess)), locations, k)
        #Y = assign_facilities() <-- do this in the algorithm?
        obj_value = calculate_objective(distances, X, Y)
        covered_dict[set_guess] = obj_value

covered_dict

[[0, 1, 4], [0, 2, 3], [0, 2, 4], [1, 1, 3], [1, 1, 4], [1, 2, 3], [1, 2, 4]]


In [153]:
'''c ← first(P)
while c ≠ Λ do
    if valid(P,c) then
        output(P, c)
    c ← next(P, c)
end while'''

from itertools import combinations

client_locations = [[0, 1],[1,2],[3, 4, 5]]

client_locations_excluded = []
for person in client_locations:
    if len(person)>1:
        client_locations_excluded.append(person[1:])

possible_hits = set(i for loc_set in client_locations_excluded for i in loc_set)
combo = combinations(possible_hits, len(possible_hits))

print(combo)

[(1, 2, 4, 5)]


In [255]:
def fpt(G: List[List[float]], client_locations: List[List[int]], k):
    
    """
    Remove homes from the client_location list
    Perhaps create mapping for the indices of people before exclusion and after?
    """
    client_locations_excluded = []
    for person in client_locations:
        if len(person)>1:
            client_locations_excluded.append(person[1:])
    
    guesses = [([loc[0] for loc in client_locations_excluded], [0 for i in range(len(client_locations_excluded))])]

    covered_set = set()
    while len(guesses)>0:
        guess, indexing = guesses.pop(0)
        print(guess)

        """
        Locate facilities based on guess
        Reformat input to work for k-supplier
        """
        '''locations = list(set(k for i in client_locations_excluded for k in i))
        facilities = _k_supplier(G, guess, locations, k)
        Y = assign_facilities(G, client_locations_excluded, facilities)
        obj_value = calculate_objective(G, Y)'''
        
        """
        Add more guess combinations by swapping out locations
        Can alter this to reduce number of repeats?
        Change "covered_set" to record based on indexing? (hash)
        """
        for person, current_loc_ind in enumerate(indexing):
            if current_loc_ind+1 < len(client_locations_excluded[person]):
                new_indexing = [indexing[i] if i!=person else indexing[i]+1 for i in range(len(indexing))]
                new_guess = [locations[new_indexing[ind]] for ind, locations in enumerate(client_locations_excluded)]
                if frozenset(new_guess) not in covered_set:
                    guesses.append((new_guess, new_indexing))
                    #covered_set.add(frozenset(new_guess))
    

In [264]:
"""
COMBINATIONS REDONE
"""
def fpt(G: List[List[float]], client_locations: List[List[int]], k):
    
    """
    Remove homes from the client_location list
    Perhaps create mapping for the indices of people before exclusion and after?
    """
    client_locations_excluded = []
    for person in client_locations:
        if len(person)>1:
            client_locations_excluded.append(person[1:])
    
    guesses:List[Tuple[List[int], List[int], int]]
    guesses = [([loc[0] for loc in client_locations_excluded], [0 for i in range(len(client_locations_excluded))], 0)]

    covered_set = set()
    objective_facilities: List[Tuple[int, List[int], List[int]]] = []
    while len(guesses)>0:
        guess, indexing, person = guesses.pop(0)

        """
        Locate facilities based on guess
        Reformat input to work for k-supplier
        """
        locations = list(set(k for i in client_locations_excluded for k in i))
        facilities = _k_supplier(G, list(set(guess)), locations, k)
        Y = assign_facilities(G, client_locations_excluded, facilities)
        obj_value = calculate_objective(G, Y)
        
        objective_facilities.append((obj_value, facilities, guess))
        """
        Add more guess combinations by swapping out locations
        """
        for ind in range(person, len(indexing)):
            current_loc_ind = indexing[ind]
            
            if current_loc_ind+1 < len(client_locations_excluded[ind]):
                new_indexing = indexing[:ind] + [current_loc_ind+1] + indexing[ind+1:]
                new_guess = [locations[new_indexing[i]] for i, locations in enumerate(client_locations_excluded)]
                if frozenset(new_guess) not in covered_set:
                    guesses.append((new_guess, new_indexing, ind))
                    covered_set.add(frozenset(new_guess))
    print(sorted(objective_facilities))

In [266]:
fpt([[0],[1, 0],[2, 1, 0],[3, 2, 1, 0]], [[0, 3], [1, 3]], 2)

0
{3}
[(0, [3], [3, 3])]
